In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import scipy
from sympy.matrices import Matrix, eye
#
from SBMLLint.games.message import Message

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [55]:
simple = load_file_from_curated_data(478)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
    print(r.makeIdentifier(is_include_kinetics=False))

R0: Ras2_GDP + Cdc25 -> Ras2_GDP_Cdc25
R1: Ras2_GDP_Cdc25 -> Ras2_GDP + Cdc25
R2: Ras2_GDP_Cdc25 -> Ras2_Cdc25 + GDP
R3: Ras2_Cdc25 + GDP -> Ras2_GDP_Cdc25
R4: Ras2_Cdc25 + GTP -> Ras2_GTP_Cdc25
R5: Ras2_GTP_Cdc25 -> Ras2_Cdc25 + GTP
R6: Ras2_GTP_Cdc25 -> Cdc25 + Ras2_GTP
R7: Cdc25 + Ras2_GTP -> Ras2_GTP_Cdc25
R8: Ras2_GTP + Ira2 -> Ras2_GTP_Ira2
R9: Ras2_GTP_Ira2 -> Ras2_GDP + Ira2
R10: Ras2_GTP + CYR1 -> Ras2_GTP_CYR1
R11: Ras2_GTP_CYR1 + ATP -> Ras2_GTP_CYR1 + cAMP
R12: Ira2 + Ras2_GTP_CYR1 -> Ras2_GDP + Ira2 + CYR1
R13: cAMP + PKA -> cAMP_PKA
R14: cAMP + cAMP_PKA -> IIcAMP_PKA
R15: cAMP + IIcAMP_PKA -> IIIcAMP_PKA
R16: cAMP + IIIcAMP_PKA -> IVcAMP_PKA
R17: IVcAMP_PKA -> cAMP + IIIcAMP_PKA
R18: IIIcAMP_PKA -> cAMP + IIcAMP_PKA
R19: IIcAMP_PKA -> cAMP + cAMP_PKA
R20: cAMP_PKA -> cAMP + PKA
R21: IVcAMP_PKA -> 2.00 C + 2.00 R_2cAMP
R22: R_2cAMP -> 2.00 cAMP + R
R23: C + R -> R_C
R24: 2.00 R_C -> PKA
R25: C + Pde1 -> C + Pde1f
R26: cAMP + Pde1f -> cAMP_Pde1f
R27: cAMP_Pde1f -> cAMP + Pd

In [56]:
m = Message(simple)
m

{Ras2_GDP};{Cdc25};{PPA2};{PKA};{R_C};{GTP};{ATP};{Ras2_GTP_Ira2};{Ras2_GDP_Cdc25};{Ira2};{IIcAMP_PKA};{cAMP};{cAMP_Pde2};{C};{Ira2P};{Pde2};{R_2cAMP};{GDP};{Ras2_GTP_Ira2P};{IIIcAMP_PKA};{Cdc25f};{Pde1};{Ras2_Cdc25};{Ras2_GTP};{Ras2_GTP_CYR1};{Pde1f};{AMP};{R};{cAMP_Pde1f};{CYR1};{IVcAMP_PKA};{cAMP_PKA};{Ras2_GTP_Cdc25}

In [57]:
m.analyze()

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  []
Type III error:  []
Type I-SOM error:  set()


False

In [45]:
m.in_edges()

re24: 2.00 {NADH} + 2.00 {F6P, G6P} -> 2.00 {NAD} + 2.00 {Mannitol, Mannitol1Phosphate, Mannitol_Ext}
reaction_n_n
re14: 0.67 {Acetoin, Acetoin_Ext} + 1.33 {Mannitol, Mannitol1Phosphate, Mannitol_Ext} -> 2.00 {Pint} + 2.00 {F6P, G6P}
reaction_n_n
re18: 0.50 {Acetoin, Acetoin_Ext} -> {BPG}
reaction_1_n
re26: 2.00 {PYR} -> {Acetoin, Acetoin_Ext}
reaction_n_1
re17: 0.33 {Acetoin, Acetoin_Ext} + 0.67 {Mannitol, Mannitol1Phosphate, Mannitol_Ext} -> 2.00 {G3P}
reaction_n_n
re12: {Glucose} -> {F6P, G6P}
reaction_1_1
re23: {CoA} + {Acetate} + {Acetoin, Acetoin_Ext} + 2.00 {Mannitol, Mannitol1Phosphate, Mannitol_Ext} -> {AcetCoA} + 3.00 {F6P, G6P}
reaction_n_n
re33: {FBP} -> 0.33 {Acetoin, Acetoin_Ext} + 0.67 {Mannitol, Mannitol1Phosphate, Mannitol_Ext}
reaction_n_n
re22: 0.50 {Acetoin, Acetoin_Ext} + {Mannitol, Mannitol1Phosphate, Mannitol_Ext} -> {Lactate} + {F6P, G6P}
reaction_n_n
re20: 0.75 {Pext} + 1.50 {F6P, G6P} -> 0.50 {Acetoin, Acetoin_Ext} + {Mannitol, Mannitol1Phosphate, Mannitol_Ext

In [19]:
m.getNode(m.simple.getMolecule('CamT_Ca3_ABD'))

{CamR_Ca3_ABD, CamT_Ca3_ABD}

In [19]:
m.get_edge_data(s21, s13)

{'reaction': ['reaction_25']}

In [14]:
m.echelon_df['reaction_22']

{species_19}    0.0
{species_6}     0.0
{species_12}    0.0
{species_15}    0.0
{species_22}    1.0
{species_3}     0.0
{species_8}     0.0
{species_10}    0.0
{species_9}     0.0
{species_18}    0.0
{species_2}     0.0
{species_20}    0.0
{species_16}    0.0
{species_14}    0.0
{species_17}    0.0
{species_11}    0.0
{species_23}    0.0
{species_1}     0.0
{species_5}     0.0
{species_4}     0.0
{species_13}    1.0
{species_21}   -1.0
{species_7}     0.0
Name: reaction_22, dtype: float64

In [15]:
simple.getReaction('reaction_22')

reaction_22: species_21 -> species_13 + species_22; compartment_1 * k1 * species_21

In [16]:
simple.getReaction('reaction_25')

reaction_25: species_22 -> 3.00 species_8 + species_23; compartment_1 * k1 * species_22

In [30]:
m.som_reactions_lu[14]

reaction_25: {species_22} -> 3.00 {species_8} + {species_23}

In [41]:
m.get_edge_data(s13, s21)

{'reaction': ['reaction_22', 'reaction_25']}

In [40]:
m.get_edge_data(s21, s13)

{'reaction': ['reaction_25']}

In [42]:
m.som_reactions_lu

[reaction_3: {species_4} -> 2.00 {species_5},
 reaction_6: {species_9} -> {species_3} + {species_4},
 reaction_8: {species_10} -> {species_7} + {species_9},
 reaction_9: {species_10} -> {species_6} + {species_11},
 reaction_10: {species_11} -> {species_12} + {species_4},
 reaction_13: {species_14} -> {species_13} + {species_12},
 reaction_15: {species_16} -> {species_14} + {species_15},
 reaction_16: {species_16} -> {species_18} + {species_17},
 reaction_17: {species_17} -> {species_13} + {species_19},
 reaction_20: {species_20} -> {species_17} + {species_15},
 reaction_21: {species_20} -> {species_18} + {species_21},
 reaction_22: {species_21} -> {species_13} + {species_22},
 reaction_23: {species_12} -> {species_8} + {species_23},
 reaction_24: {species_19} -> 2.00 {species_8} + {species_23},
 reaction_25: {species_22} -> 3.00 {species_8} + {species_23},
 reaction_1: {species_1} + {species_2} -> {species_3},
 reaction_2: 2.00 {species_5} -> {species_4},
 reaction_4: {species_8} + {sp

In [41]:
echelon[colname][::-1]

{Br, Ce}    0.0
{BrO3}      0.0
{HOBr}      0.0
{HBrO2}     0.0
Name: Reaction1, dtype: float64

In [77]:
rref_operation = np.identity(echelon_t.shape[0])
rref_operation = pd.DataFrame(rref_operation,
                  index = echelon_t.index,
                  columns = echelon_t.index)
rref_operation

,Reaction4,Reaction2,Reaction3,Reaction1
Reaction4,1.0,0.0,0.0,0.0
Reaction2,0.0,1.0,0.0,0.0
Reaction3,0.0,0.0,1.0,0.0
Reaction1,0.0,0.0,0.0,1.0


In [116]:
rref_operation = np.identity(echelon_t.shape[0])
rref_operation = pd.DataFrame(rref_operation,
                  index = echelon_t.index,
                  columns = echelon_t.index)

# now update the operation matrix, finally multiply (dot product) two matrices
echelon = m.echelon_df
for idx, colname in enumerate(echelon.columns):
  reaction_series = echelon[colname]
#   # find the first nonzero values
  nonzero_idx = np.nonzero(echelon[colname])[0]
  # Skip if there is no nonzero value or if it is first reaction
  if not nonzero_idx.any() or idx == 0:
    continue
  nonzero_species = reaction_series.index[nonzero_idx[0]]
  nonzero_value = reaction_series[nonzero_idx[0]]
  # find current nonzero index
  current_echelon = rref_operation.dot(echelon.T).T
  for prev_colname in current_echelon.columns[:idx]:
    print("Colname is: ", colname)
    print("Previous columns are:", prev_colname)
    if np.round(current_echelon[prev_colname][nonzero_species], 3) != 0.0:
      reduction_value = current_echelon[prev_colname][nonzero_species]
      print("We need to reduce ", reduction_value)
      rref_operation.at[prev_colname, colname] = (-1.0) * reduction_value / nonzero_value
rref_operation

Colname is:  Reaction2
Previous columns are: Reaction4
We need to reduce  1.0
Colname is:  Reaction3
Previous columns are: Reaction4
We need to reduce  1.3333333333333333
Colname is:  Reaction3
Previous columns are: Reaction2
We need to reduce  -0.5


,Reaction4,Reaction2,Reaction3,Reaction1
Reaction4,1.0,-0.666667,4.0,0.0
Reaction2,0.0,1.000000,-1.5,0.0
Reaction3,0.0,0.000000,1.0,0.0
Reaction1,0.0,0.000000,0.0,1.0


In [120]:
np.round(rref_operation.dot(echelon_t), 3)

,{HBrO2},{HOBr},{BrO3},"{Br, Ce}"
Reaction4,-2.0,0.0,-0.000,6.000
Reaction2,0.0,1.5,-0.000,-3.000
Reaction3,0.0,0.0,-0.333,1.333
Reaction1,0.0,0.0,0.000,0.000


In [121]:
rref_operation.dot(m.lower_inverse)

,Reaction4,Reaction2,Reaction3,Reaction1
Reaction4,4.000000,-2.000000,4.0,0.0
Reaction2,-1.500000,1.500000,-1.5,0.0
Reaction3,0.666667,-0.333333,1.0,0.0
Reaction1,1.000000,-1.000000,0.0,1.0


In [108]:
rref_op1 = np.identity(echelon_t.shape[0])
rref_op1 = pd.DataFrame(rref_op1,
          index = echelon_t.index,
          columns = echelon_t.index)
val = 1/1.5*(-1.0)
rref_op1.at['Reaction4', 'Reaction2'] = val
op1 = rref_op1.dot(echelon_t)
op1
rref_op1

,Reaction4,Reaction2,Reaction3,Reaction1
Reaction4,1.0,-0.666667,0.0,0.0
Reaction2,0.0,1.000000,0.0,0.0
Reaction3,0.0,0.000000,1.0,0.0
Reaction1,0.0,0.000000,0.0,1.0


In [111]:
op1

,{HBrO2},{HOBr},{BrO3},"{Br, Ce}"
Reaction4,-2.0,5.551115e-17,1.333333,0.666667
Reaction2,0.0,1.500000e+00,-0.500000,-1.000000
Reaction3,0.0,0.000000e+00,-0.333333,1.333333
Reaction1,0.0,0.000000e+00,0.000000,0.000000


In [110]:
rref_op2 = np.identity(echelon_t.shape[0])
rref_op2 = pd.DataFrame(rref_op2,
          index = echelon_t.index,
          columns = echelon_t.index)
val = op1.loc['Reaction4', '{BrO3}'] / op1.loc['Reaction3', '{BrO3}'] * (-1.0)
rref_op2.at['Reaction4', 'Reaction3'] = val
op2 = rref_op2.dot(op1)
op2
rref_op2

,Reaction4,Reaction2,Reaction3,Reaction1
Reaction4,1.0,0.0,4.0,0.0
Reaction2,0.0,1.0,0.0,0.0
Reaction3,0.0,0.0,1.0,0.0
Reaction1,0.0,0.0,0.0,1.0


In [113]:
op2

,{HBrO2},{HOBr},{BrO3},"{Br, Ce}"
Reaction4,-2.0,5.551115e-17,-7.401487e-17,6.000000
Reaction2,0.0,1.500000e+00,-5.000000e-01,-1.000000
Reaction3,0.0,0.000000e+00,-3.333333e-01,1.333333
Reaction1,0.0,0.000000e+00,0.000000e+00,0.000000


In [106]:
rref_op1

,Reaction4,Reaction2,Reaction3,Reaction1
Reaction4,1.0,-0.666667,1.0,0.0
Reaction2,0.0,1.000000,-1.5,0.0
Reaction3,0.0,0.000000,1.0,0.0
Reaction1,0.0,0.000000,0.0,1.0


In [103]:
rref_op2

,Reaction4,Reaction2,Reaction3,Reaction1
Reaction4,1.0,-0.666667,1.0,0.0
Reaction2,0.0,1.000000,-1.5,0.0
Reaction3,0.0,0.000000,1.0,0.0
Reaction1,0.0,0.000000,0.0,1.0


In [95]:
np.round(rref_operation.dot(echelon_t), 3)

,{HBrO2},{HOBr},{BrO3},"{Br, Ce}"
Reaction4,-2.0,0.0,0.333,4.667
Reaction2,0.0,1.5,-0.000,-3.000
Reaction3,0.0,0.0,-0.333,1.333
Reaction1,0.0,0.0,0.000,0.000


In [94]:
m.convertMatrixToSOMReactions(np.round(rref_operation.dot(echelon_t), 3).T)

[Reaction4: 2.00 {HBrO2} -> 0.33 {BrO3} + 4.67 {Br, Ce},
 Reaction2: 3.00 {Br, Ce} -> 1.50 {HOBr},
 Reaction3: 0.33 {BrO3} -> 1.33 {Br, Ce},
 Reaction1:  -> ]

In [90]:
rref_operation.dot(echelon_t).apply(np.round)

,{HBrO2},{HOBr},{BrO3},"{Br, Ce}"
Reaction4,-2.0,0.0,0.0,5.0
Reaction2,0.0,2.0,-0.0,-3.0
Reaction3,0.0,0.0,-0.0,1.0
Reaction1,0.0,0.0,0.0,0.0


In [11]:
m.convertMatrixToReactions(simple, m.permuted_matrix)

[ReactionSummary(label=Reaction1, reactants=[BrO3 *  1.00, Br *  1.00], products=[HBrO2 *  1.00, HOBr *  1.00], category='reaction_n_n'),
 ReactionSummary(label=Reaction3, reactants=[BrO3 *  1.00], products=[Ce *  1.00, HBrO2 *  1.00], category='reaction_1_n'),
 ReactionSummary(label=Reaction2, reactants=[Br *  1.00, HBrO2 *  1.00], products=[HOBr *  2.00], category='reaction_n_n'),
 ReactionSummary(label=Reaction5, reactants=[Ce *  1.00], products=[Br *  1.00], category='reaction_1_1'),
 ReactionSummary(label=Reaction4, reactants=[HBrO2 *  2.00], products=[BrO3 *  1.00, HOBr *  1.00], category='reaction_n_n')]

In [12]:
m.analyze()

We just analyzed the data...
RREF error:  []
Type I error:  []
Type II error:  []


True

In [13]:
m

{Ce}->{HOBr}
{Br}->{HOBr}
{HOBr}->{HBrO2}
{BrO3}

In [32]:
# mat_df = m.stoichiometry_matrix
# mat_t = mat_df.T

# mat_t
# idx_mat_t = mat_t.index
# # # LU decomposition
# mat_lu = scipy.linalg.lu(mat_t)
# # # inverse pivot matrix
# p_inv = scipy.linalg.inv(mat_lu[0])
# pivot_index = [list(k).index(1) for k in p_inv]
# new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
# # # row reduced matrix
# row_reduced = pd.DataFrame(mat_lu[2], index=new_idx_mat_t, columns=mat_t.columns)
# row_reduced

In [125]:
rref = [1]
er = []
if rref and not er:
  print("haha")

haha
